<a href="https://colab.research.google.com/github/MoussaNEYA/Yolo_detect_Object_model/blob/main/Train_YOLO_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Entrainement avec Google Colab

**Verification de la validité NVIDIA GPU**

In [ ]:
!nvidia-smi

## 1.&nbsp;Rassembler et étiqueter les images de formation

## 2.&nbsp;Télécharger l'ensemble de données d'image et préparer les données de formation

Ensuite, nous allons importer notre jeu de données et le préparer pour l'entraînement avec YOLO. Nous allons diviser le jeu de données en dossiers d'entraînement et de validation, et générer automatiquement le fichier de configuration pour l'entraînement du modèle.

## 2.1 Téléchargement d'images
Tout d'abord, nous devons télécharger le jeu de données sur Colab.

**Option 1. Télécharger via Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/path/to/data.zip /content

Option 2.Télécharger en local en chargeant les données sous format .zip

## 2.2 Diviser les images en dossiers d'entrainement et de validation

In [ ]:
# Décompressez les images dans un dossier de données personnalisé
!unzip -q /content/data.zip -d /content/custom_data

Ultralytics nécessite une structure de dossiers particulière pour stocker les données d'entraînement des modèles. Le dossier racine est nommé « data ». Il contient deux dossiers principaux :
* **Train** : Ce sont les images utilisées pour entraîner le modèle. Lors d'une période d'entraînement, chaque image de l'ensemble d'entraînement est transmise au réseau de neurones. L'algorithme d'entraînement ajuste les pondérations du réseau pour correspondre aux données des images.
* **Validation** : Ces images servent à vérifier les performances du modèle à la fin de chaque période d'entraînement.
Chacun de ces dossiers contient un dossier « images » et un dossier « labels », qui contiennent respectivement les fichiers image et les fichiers d'annotation.

J'ai écrit un script Python qui crée automatiquement la structure de dossiers requise et déplace aléatoirement 90 % de l'ensemble de données vers le dossier « train » et 10 % vers le dossier « validation ». Exécutez le bloc de code suivant pour exécuter le script.

In [ ]:
# À faire : améliorer la robustesse du script train_val_split.py afin qu'il puisse gérer les dossiers de données imbriqués, etc.
!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

## 3. Configuration requise pour l'installation (Ultralytics)
Nous allons ensuite installer la bibliothèque Ultralytics dans cette instance Google Colab. Cette bibliothèque Python servira à entraîner le modèle YOLO.

In [ ]:
!pip install ultralytics

## 4.&nbsp;Configuré l'entrainement

In [ ]:
# Fonction Python pour créer automatiquement le fichier de configuration data.yaml
# 1. Lecture du fichier « classes.txt » pour obtenir la liste des noms de classes
# 2. Création d'un dictionnaire de données avec les chemins d'accès corrects aux dossiers, le nombre de classes et leurs noms
# 3. Écriture des données au format YAML dans data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

# Lecture du fichier « classes.txt » pour obtenir la liste des noms de classes
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

# Créer un dictionnaire de données
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

# Écriture des données au format YAML dans data.yaml
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Définir le dossier de classes.txt et exécuter function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

## 5.&nbsp;Entrainement du Model

## 5.1 Paramètre de l'entrainement
**Architecture et taille du modèle (`model`) :**
Vous pouvez également entraîner les modèles `YOLOv8` ou `YOLOv5` en remplaçant `yolov8` ou `yolov5` par `yolo11`.

**Nombre d'époques (`epochs`)**

**Résolution (`imgsz`)**

## 5.2 Lancement de l'entrainement!

Exécutez le bloc de code suivant pour commencer l'entraînement. Si vous souhaitez utiliser un modèle, un nombre d'époques ou une résolution différents, modifiez `model`, `epochs` ou `imgsz`.

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

**NB :** Il faut permettre à la formation de s'exécuter jusqu'à son terme, car un optimiseur s'exécute à la fin de la formation et supprime les couches inutiles du modèle.

## 6.&nbsp;Test du Model

In [ ]:
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

**Pour afficher les 10 première données de test**

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')


## 7.&nbsp;Deploiyement du Model

## 7.1 Téléchargement de notre model YOLO

In [ ]:
# Creation de "my_model" dossier pour stocker les poids des modèles et résultats entraînement
!mkdir /content/my_model
!cp /content/runs/detect/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/detect/train /content/my_model

# Pour zipper notre model "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
# Téléchargement du model en local
from google.colab import files

files.download('/content/my_model.zip')

## 7.2 Déploiyement du moedel Yolo en local 

## 7.2.1 Deployer sur mon PC
**1. Télécharger et installé Anaconda ou Miniconda (c'est ce que je vous recommande le plus)**

**2. Créer un environnement virtuelle**

```
conda create --name yolo-env1 python=3.12 -y
conda activate yolo-env1
```
**Installé Ultralytics :**
```
pip install ultralytics
```
**Si vous disposez d'un GPU NVIDIA, vous pouvez installer la version compatible GPU de PyTorch en exécutant la commande suivante :**
```
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
```
**3. Extraction du model**
il s'agit ddézipper le fichier `my_model.zip` télécharger à l'etape 7.1 précédent

```
cd emplacement/du/dossier
```
**4. Déplacer le script déexecution yolo_detect.py dans le dossier dézipper précedement**

Très bien ! Nous sommes prêts à exécuter le script. Pour exécuter l'inférence avec un modèle `yolov8s` sur une caméra USB en résolution 1280 x 720, exécutez:
```
python yolo_detect.py --model my_model.pt --source usb0 --resolution 1280x720
```
Une fenêtre apparaîtra affichant un flux en direct de votre webcam avec des cadres dessinés autour des objets détectés dans chaque image.

```
Pour plus de détaille taper python yolo_detect.py --help

```

## 8.&nbsp;Conclusion

Félicitations! à nous; qui venions d'entraîné et déployé avec succès un modèle de détection d'objets YOLO. 😀